# EIS

In [ ]:
# 字段映射关系
column_mapping = {
    "Data_Point": "Data Point",
    "Test_Time(s)": "Test Time (s)",
    "Date_Time": "Date Time",
    "Step_Time(s)": "Step Time (s)",
    "Step_Index": "Step Index",
    "Cycle_Index": "Cycle Index",
    "Current(A)": "Current (A)",
    "Voltage(V)": "Voltage (V)",
    "Charge_Capacity(Ah)": "Charge Capacity (Ah)",
    "Discharge_Capacity(Ah)": "Discharge Capacity (Ah)",
    "Charge_Energy(Wh)": "Charge Energy (Wh)",
    "Discharge_Energy(Wh)": "Discharge Energy (Wh)",
    "dV/dt(V/s)": "dV/dt (V/s)",
    "Internal_Resistance(Ohm)": "Internal Resistance (Ohm)",
    "Is_FC_Data": "Is FC Data",
    "AC_Impedance(Ohm)": "AC Impedance (Ohm)",
    "ACI_Phase_Angle(Deg)": "ACI Phase Angle (Deg)",
    "Temperature (C)_1": "Temperature (C)_1",
    "Temperature (C)_2": "Temperature (C)_2",
    "Time_sec": "Time_sec",
    "Current_Amp": "Current_Amp",
    "Voltage_Volt": "Voltage_Volt",
    "Charge_Ah": "Charge_Ah",
    "Discharge_Ah": "Discharge_Ah",
    "Frequency (Hz)": "Frequency (Hz)",
    "Z' (Ω)": "Z' (Ohm)",
    "-Z'' (Ω)": "-Z'' (Ohm)",
    "Z (Ω)": "Z (Ohm)",
    "-Phase (°)": "-Phase (Deg)",
    "Freq(Hz)": "Freq (Hz)",
    "Ampl": "Ampl",
    "Bias": "Bias",
    "Time(Sec)": "Time (Sec)",
    "Z'(a)": "Z'(a)",
    "Z''(b)": "Z''(b)",
    "GD": "GD",
    "Err": "Err",
    "Range": "Range",
    "SOC": "SOC",
    "Real Part of Impedance": "Real Part of Impedance",
    "Imaginary Part of Impedance": "Imaginary Part of Impedance",
    "Cell Temperature": "Cell Temperature",
    "Temperature at Cell Connector": "Temperature at Cell Connector"
}

all_cols = list(column_mapping.values())

In [ ]:
from keras.utils import  to_categorical
from keras.models import Model
from keras.layers import Flatten, Dense,Input
from sklearn.metrics import confusion_matrix, roc_curve, auc,f1_score
from utils import calculate_metrics
from keras.optimizers import Adam
from KAN import KANLinear
from TransformerEncoer import Encoder
from keras.losses import categorical_crossentropy
import tensorflow as tf
import numpy as np
from keras.callbacks import Callback


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']   #显示中文
plt.rcParams['axes.unicode_minus']=False       #显示负号
plt.rcParams['font.family'] = 'Times New Roman'
import warnings
warnings.filterwarnings("ignore")
import os

In [ ]:
battery_material_dict = {
    0: 'NCR18650B',
    1: 'unknown',
    2: 'Eunicell LR2032',
}

In [ ]:
LOOKBACK = 30

In [ ]:
def split_sequence(sequence, look_back,pre_l):
    X, y = [], []
    for i in range(len(sequence)):
        end_element_index = i + look_back
        if end_element_index > len(sequence) - 1 -pre_l:
            break
        sequence_x, sequence_y = sequence[i:end_element_index], sequence[end_element_index:end_element_index+pre_l]
        X.append(sequence_x)
        y.append(sequence_y)

    return np.array(X).astype('float32'),np.array(y)

In [ ]:
SANDIA_X = []
SANDIA_Y = []

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
def SANDIA_PARSE(SANDIA_LCO_files,tearget=1):
    for fn in SANDIA_LCO_files:
        empty_df = pd.DataFrame(columns=all_cols)
        try:
            with open(fn,'r') as f:
                if '_Pristine_' in fn:
                    eis_data = f.readlines()[1:]
                else:
                    eis_data = f.readlines()[123:]
            with open('temp','w') as f:
                f.writelines(eis_data)
            if '_Pristine_' in fn:
                eis_data_df = pd.read_csv('temp',header=None,names=['index','Frequency (Hz)',"Z' (Ω)","-Z'' (Ω)",'Z (Ω)','-Phase (°)','Time (s)'])
                empty_df['Frequency (Hz)'] = eis_data_df['Frequency (Hz)'].values
                empty_df['Z\' (Ohm)'] = eis_data_df['Z\' (Ω)'].values
                empty_df['-Z\'\' (Ohm)'] = eis_data_df['-Z\'\' (Ω)'].values
                empty_df['-Phase (Deg)'] = eis_data_df['-Phase (°)'].values
                empty_df = empty_df.fillna(0)
                eis_data = empty_df.values
            else:
                eis_data_df = pd.read_csv('temp',header=None,names=['Freq(Hz)','Ampl',"Bias","Time(Sec)","Z'(a)","Z''(b)",'GD','Err','Range'])
                empty_df['Frequency (Hz)'] = eis_data_df['Freq(Hz)'].values
                empty_df['Z\' (Ohm)'] = eis_data_df["Z'(a)"].values
                empty_df['-Z\'\' (Ohm)'] = eis_data_df["Z''(b)"].values
                empty_df = empty_df.fillna(0)
                eis_data = empty_df.values
            xx,_ = split_sequence(scaler.fit_transform(eis_data),LOOKBACK,1)
            SANDIA_X.append(xx)
            SANDIA_Y.append(np.full(len(xx),tearget))
        except: continue

In [ ]:

SANDIA_LCO_files = ['./data/Sandia/EIS_Data/'+ i for i in os.listdir('./data/Sandia/EIS_Data') if i.startswith('LCO')]
SANDIA_PARSE(SANDIA_LCO_files,0)
SANDIA_LCO_files = ['./data/Sandia/EIS_Data/'+ i for i in os.listdir('./data/Sandia/EIS_Data') if i.startswith('LFP')]
SANDIA_PARSE(SANDIA_LCO_files,0)
SANDIA_LCO_files = ['./data/Sandia/EIS_Data/'+ i for i in os.listdir('./data/Sandia/EIS_Data') if i.startswith('NMC')]
SANDIA_PARSE(SANDIA_LCO_files,0)
SANDIA_LCO_files = ['./data/Sandia/EIS_Data/'+ i for i in os.listdir('./data/Sandia/EIS_Data') if i.startswith('NCA')]
SANDIA_PARSE(SANDIA_LCO_files,0)

In [ ]:
ZHANG = pd.read_csv('./data/Zhang/EIS data/EIS data/EIS_state_IX_25C08.txt',sep='\t')
empty_df = pd.DataFrame(columns=all_cols)
empty_df['Frequency (Hz)'] = ZHANG['freq/Hz'].values
empty_df["Z\'\'(b)"] = ZHANG['           Re(Z)/Ohm'].values
empty_df["Z\' (Ohm)"] = ZHANG['  -Im(Z)/Ohm'].values
empty_df["Z (Ohm)"] = ZHANG['   |Z|/Ohm'].values
empty_df["-Phase (Deg)"] = ZHANG['   Phase(Z)/deg'].values
ZHANG =empty_df.iloc[1000:2000,:]
ZHANG = ZHANG.fillna(0)

In [ ]:
SCIWELL_files = ['./data/SCIWELL/cell_checkup/cell_checkups/EIS/' + i for i in os.listdir('./data/SCIWELL/cell_checkup/cell_checkups/EIS') if i.endswith('.csv')]
SCIWELL = pd.read_csv(SCIWELL_files[0]).loc[:,['Frequency [Hz]','Real Part of Impedance [Ohm]','Imaginary Part of Impedance [Ohm]']]
empty_df = pd.DataFrame(columns=all_cols)
empty_df['Frequency (Hz)'] = SCIWELL['Frequency [Hz]'].values
empty_df['Real Part of Impedance'] = SCIWELL['Real Part of Impedance [Ohm]'].values
empty_df['Imaginary Part of Impedance'] = SCIWELL['Imaginary Part of Impedance [Ohm]'].values
SCIWELL = empty_df.fillna(0)

In [ ]:
all_X = []
all_Y = []

SCIWELL_v = SCIWELL.values
xx,_ = split_sequence(SCIWELL_v/(np.max(SCIWELL_v,axis=0)),30,pre_l=1)
all_X.append(xx)
all_Y.append(np.full((xx.shape[0],1),1))

ZHANG_v = ZHANG.values
xx,_ = split_sequence(ZHANG_v/np.max(ZHANG_v,axis=0),LOOKBACK,pre_l=1)
all_X.append(xx)
all_Y.append(np.full((xx.shape[0],1),2))


all_X = np.concatenate(all_X,axis=0)
all_Y = np.concatenate(all_Y,axis=0)


In [ ]:
SANDIA_X = np.concatenate(SANDIA_X, axis=0)
SANDIA_Y = np.array(SANDIA_Y)
SANDIA_Y = np.concatenate(SANDIA_Y,axis=0)
all_X = np.concatenate([all_X, SANDIA_X])
all_Y = np.concatenate([all_Y, np.expand_dims(SANDIA_Y, axis=1)])
all_Y = to_categorical(all_Y,3)


In [ ]:

class AdpWeightedLoss():
    def __init__(self):
        self.class_weights = tf.Variable(tf.ones(shape=(4,)), trainable=True)

    def weighted_categorical_crossentropy(self,y_true, y_pred):
        cross_entropy = categorical_crossentropy(y_true, y_pred)
        y_true_idx = tf.argmax(y_true, axis=-1)
        weights = tf.gather(self.class_weights, y_true_idx)
        positive_weights = tf.nn.softplus(weights)
        weighted_loss = cross_entropy * positive_weights
        return weighted_loss
    
    def __call__(self, y_true, y_pred):
        return self.weighted_categorical_crossentropy(y_true, y_pred)

def get_model():
    inp = Input((30,43))
    temp = Dense(80, activation='relu')(inp)
    ts_encoder = Encoder(d_model=80,num_heads=4,seql=30,dropout_rate=0,N_block=1,name='encoder')
    encoder_outputs = ts_encoder(temp) # x,lstm_pe_embeding,mha_results_list,attn_results_list
    temp = encoder_outputs[0]  # 只取第一个输出 
    temp = Flatten()(temp)
    kan = KANLinear(128)
    temp = kan(temp)
    temp = KANLinear(80)(temp)
    temp = Dense(64,activation='relu')(temp)
    out = Dense(3,activation='softmax')(temp)
    model = Model(inp, out)
    return model,kan



class F1Callback(Callback):
    def __init__(self, validation_data=(), patience=20, fold=0):
        super(F1Callback, self).__init__()
        self.patience = patience
        self.X_val, self.y_val = validation_data
        self.best_f1 = 0
        self.wait = 0
        self.filepath = f'./weights/EIS_Trans_KAN_models_fold_{fold}.weights.h5'

    def on_epoch_end(self, epoch, logs={}):
        # 预测验证集
        y_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        y_val = np.argmax(self.y_val, axis=1)
        
        # 计算F1分数
        current_f1 = f1_score(y_val, y_pred, average='weighted')
        print(f'\nValidation F1 Score: {current_f1:.4f}')
        
        # 如果当前F1分数比之前的好，则保存模型
        if current_f1 > self.best_f1:
            print(f'\nF1 improved from {self.best_f1:.4f} to {current_f1:.4f}, saving model to {self.filepath}')
            self.best_f1 = current_f1
            self.wait = 0
            self.model.save_weights(self.filepath)
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f'\nEarly stopping: F1 has not improved for {self.patience} epochs.')
                self.model.stop_training = True

In [ ]:
from sklearn.model_selection import KFold
from his_plot import plot_training_history
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_preds = []
all_reals = []
all_y_scores = []
for fold, (train_index, test_index) in enumerate(kf.split(all_X), 1):
    print(f'************************** Fold {fold} ***************************')
    
    x_train, x_test = all_X[train_index], all_X[test_index]
    y_train, y_test = all_Y[train_index], all_Y[test_index]
    
    # 处理 NaN
    X_train = np.nan_to_num(x_train, nan=0)
    X_test = np.nan_to_num(x_test, nan=0)
    y_train = np.nan_to_num(y_train, nan=0)
    y_test = np.nan_to_num(y_test, nan=0)

    # 获取模型
    model, kan = get_model()
    adploss = AdpWeightedLoss()

    # 创建回调，传入当前 fold 信息
    f1_callback = F1Callback(validation_data=(X_test, y_test), patience=100, fold=fold)
    f1_callback.k_flod_number = fold

    # 用于保存当前 fold 的所有训练历史
    all_histories = []

    # ------------------ STEP 1 ------------------
    print('STEP 1')
    f1_callback.wait = 0
    model.compile(loss=adploss, optimizer=Adam(0.001), metrics=['accuracy'])
    history1 = model.fit(X_train, y_train,
                         epochs=20,
                         batch_size=64,
                         validation_data=(X_test, y_test),
                         callbacks=[f1_callback],
                         verbose=1)
    all_histories.append(history1)

    # ------------------ STEP 2 ------------------
    print('STEP 2')
    f1_callback.wait = 0
    model.compile(loss=adploss, optimizer=Adam(0.0005), metrics=['accuracy'])
    history2 = model.fit(X_train, y_train,
                         epochs=20,
                         batch_size=64,
                         validation_data=(X_test, y_test),
                         callbacks=[f1_callback],
                         verbose=1)
    all_histories.append(history2)

    # ------------------ STEP 3 ------------------
    print('STEP 3')
    f1_callback.wait = 0
    model.compile(loss=adploss, optimizer=Adam(0.0001), metrics=['accuracy'])
    history3 = model.fit(X_train, y_train,
                         epochs=60,
                         batch_size=64,
                         validation_data=(X_test, y_test),
                         callbacks=[f1_callback],
                         verbose=1)
    all_histories.append(history3)

    # 合并所有history对象
    def merge_histories(histories):
        merged_history = {}
        for key in histories[0].history.keys():
            merged_history[key] = []
            for history in histories:
                merged_history[key].extend(history.history[key])
        return merged_history

    # 创建合并后的history对象
    merged_history = merge_histories(all_histories)
    plot_training_history(merged_history)
    model.load_weights(f1_callback.filepath)
    y_pred = model.predict(X_test)
    y_scores = y_pred.copy()
    all_y_scores.append(y_scores)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    all_preds.append(y_pred_classes)
    all_reals.append(y_test_classes)
    break

In [ ]:
labels = [battery_material_dict[i] for i in battery_material_dict.keys()]

In [ ]:
y_scores = np.concatenate(all_y_scores)
all_preds_array = np.concatenate(all_preds)
all_reals_array = np.concatenate(all_reals)


In [ ]:
# %%
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# 设置美观的样式和配色
plt.style.use('seaborn-v0_8-whitegrid')
colors = cycle(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

# 计算宏平均和微平均ROC曲线和AUC
n_classes = len(labels)

# 将真实标签二值化（如果尚未二值化）
if all_reals_array.ndim == 1:
    all_reals_bin = label_binarize(all_reals_array, classes=range(n_classes))
else:
    all_reals_bin = all_reals_array

# 计算每个类别的ROC曲线和AUC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(all_reals_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# 计算宏平均ROC曲线和AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# 计算微平均ROC曲线和AUC
fpr["micro"], tpr["micro"], _ = roc_curve(all_reals_bin.ravel(), y_scores.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# 创建宏平均ROC曲线图
plt.figure(figsize=(12, 9))
ax = plt.gca()

# 绘制宏平均ROC曲线
plt.plot(fpr["macro"], tpr["macro"],
         label=f'Macro-average ROC curve (AUC = {roc_auc["macro"]:.4f})',
         color='deeppink', linestyle=':', linewidth=4)

# 绘制每个类别的ROC曲线
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of class {labels[i]} (AUC = {roc_auc[i]:.4f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Macro-average ROC Curve', fontsize=14)

# 创建悬浮表格
table_data = [['Class', 'AUC']]
for i in range(n_classes):
    table_data.append([labels[i], f'{roc_auc[i]:.4f}'])
table_data.append(['Macro-average', f'{roc_auc["macro"]:.4f}'])
table_data.append(['Micro-average', f'{roc_auc["micro"]:.4f}'])

# 将表格添加到图中（悬浮在右上角）
col_widths = [0.15, 0.1]
table = plt.table(cellText=table_data,
                  colWidths=col_widths,
                  cellLoc='center',
                  loc='upper right',
                  bbox=[0.65, 0.1, 0.3, 0.3])  # [left, bottom, width, height]

# 设置表格样式
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# 设置单元格样式
for i in range(len(table_data)):
    for j in range(2):
        if i == 0:  # 表头
            table[(i, j)].set_facecolor('#4C72B0')
            table[(i, j)].set_text_props(weight='bold', color='white')
        elif i > 0 and i <= n_classes:  # 类别行
            table[(i, j)].set_facecolor('#f9f9f9')
        else:  # 平均行
            table[(i, j)].set_facecolor('#FFD700')
            table[(i, j)].set_text_props(weight='bold')

plt.tight_layout()
plt.show()

# 创建微平均ROC曲线图
plt.figure(figsize=(12, 9))
ax = plt.gca()

# 绘制微平均ROC曲线
plt.plot(fpr["micro"], tpr["micro"],
         label=f'Micro-average ROC curve (AUC = {roc_auc["micro"]:.4f})',
         color='deeppink', linestyle=':', linewidth=4)

# 绘制每个类别的ROC曲线
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of class {labels[i]} (AUC = {roc_auc[i]:.4f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Micro-average ROC Curve', fontsize=14)

# 创建悬浮表格（与上面相同）
table = plt.table(cellText=table_data,
                  colWidths=col_widths,
                  cellLoc='center',
                  loc='upper right',
                  bbox=[0.65, 0.1, 0.3, 0.3])

# 设置表格样式
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# 设置单元格样式
for i in range(len(table_data)):
    for j in range(2):
        if i == 0:  # 表头
            table[(i, j)].set_facecolor('#4C72B0')
            table[(i, j)].set_text_props(weight='bold', color='white')
        elif i > 0 and i <= n_classes:  # 类别行
            table[(i, j)].set_facecolor('#f9f9f9')
        else:  # 平均行
            table[(i, j)].set_facecolor('#FFD700')
            table[(i, j)].set_text_props(weight='bold')

plt.tight_layout()
plt.show()


In [ ]:
encoder_outputs_layer = model.get_layer('encoder').output
if len(encoder_outputs_layer) < 4:
    raise ValueError(f"编码器层只有{len(encoder_outputs_layer)}个输出，但需要至少4个")

custom_model = Model(
    inputs=model.input,
    outputs=encoder_outputs_layer,
    name='encoder_extracted_model'
)
encoder_outputs = custom_model.predict(X_test)
encoder_rep = encoder_outputs[0]  # 只取第一个输出 
lstm_pe_embeding = encoder_outputs[1]  # 第二个输出
mha_results_list = encoder_outputs[2]  # 第三个输出
attn_results_list = encoder_outputs[3]  # 第四个输出
real_labels = np.argmax(y_test, axis=1)
print('encoder_rep shape: ',encoder_rep.shape)
print('lstm_pe_embeding shape: ',lstm_pe_embeding.shape)
print('mha_results_list shape: ',mha_results_list[0].shape)
print('attn_results_list shape: ',attn_results_list[0].shape)
# encoder_rep shape:  (9028, 30, 80)
# lstm_pe_embeding shape:  (9028, 30, 80)
# mha_results_list shape:  (30, 80)
# attn_results_list shape:  (9028,4, 30, 30)


In [ ]:
num_classes = len(np.unique(all_reals_array))

In [ ]:
labels

In [ ]:
class_attentions = []
for class_id in range(4):
    class_indices = np.where(np.array(real_labels) == class_id)[0]
    class_attn = attn_results_list[class_indices]
    class_attentions.append(class_attn)

mean_attentions = np.zeros((4, 4, 30, 30)) 
for class_id in range(4):
    for head_id in range(4):
        mean_attentions[class_id, head_id] = np.mean(class_attentions[class_id][:, head_id, :, :], axis=0)

# 设置全局最小最大值用于统一颜色映射
vmin = np.min(mean_attentions)
vmax = np.max(mean_attentions)

# 为每个类别单独创建图表
for class_id in range(num_classes):
    fig, axes = plt.subplots(1, 4, figsize=(15, 4))
    fig.suptitle(f'Average Attention Weights for  {labels[class_id]}', fontsize=16, fontweight='bold')
    
    for head_id in range(4):
        ax = axes[head_id]
        attn_matrix = mean_attentions[class_id, head_id]
        
        # 绘制热力图
        im = ax.imshow(attn_matrix, cmap='plasma', aspect='auto', vmin=vmin, vmax=vmax)
        
        # 设置标题和标签
        ax.set_title(f'Head {head_id}', fontweight='bold')
        ax.set_xlabel('Sequence Position')
        ax.set_ylabel('Sequence Position')
        
        # 添加颜色条
        cbar = plt.colorbar(im, ax=ax, shrink=0.6)
        cbar.set_label('Attention Weight')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)

    plt.show()

In [ ]:

selected_samples = {}
for material_id, material_name in battery_material_dict.items():
    indices = np.where(real_labels == material_id)[0]
    if len(indices) > 0:
        selected_idx = np.random.choice(indices)
        selected_samples[material_name] = selected_idx
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()


# 创建自定义 colormap 对象
cmap = 'plasma' #　['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'coolwarm', 'RdYlGn', 'Spectral']

for i, (material_name, sample_idx) in enumerate(selected_samples.items()):
    data = lstm_pe_embeding[sample_idx]  # 形状: (30, 80)
    im = axes[i].imshow(data, aspect='auto', cmap=cmap, interpolation='nearest')
    axes[i].set_title(f'{material_name} (Sample {sample_idx})', fontsize=14, pad=15, fontweight='bold')
    axes[i].set_xlabel('Encoding Dimension', fontsize=12)
    axes[i].set_ylabel('Time Step', fontsize=12)
    cbar = plt.colorbar(im, ax=axes[i], shrink=0.8)
    cbar.set_label('Encoding Value', fontsize=10)
plt.tight_layout()
plt.suptitle('LSTM Position Encoding by Battery Material Type', fontsize=16, y=1.02, fontweight='bold')
plt.show()


fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, (material_id, material_name) in enumerate(battery_material_dict.items()):
    indices = np.where(real_labels == material_id)[0]
    if len(indices) > 0:
        mean_encoding = np.mean(lstm_pe_embeding[indices], axis=0)
        im = axes[i].imshow(mean_encoding, aspect='auto', cmap=cmap, interpolation='nearest')
        axes[i].set_title(f'{material_name} (Average Encoding)', fontsize=14, pad=15, fontweight='bold')
        axes[i].set_xlabel('Encoding Dimension', fontsize=12)
        axes[i].set_ylabel('Time Step', fontsize=12)
        cbar = plt.colorbar(im, ax=axes[i], shrink=0.8)
        cbar.set_label('Encoding Value', fontsize=10)
plt.tight_layout()
plt.suptitle('Average LSTM Position Encoding by Battery Material Type', fontsize=16, y=1.02, fontweight='bold')
# plt.savefig('average_lstm_position_encoding_by_material.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
selected_samples = {}
for material_id, material_name in battery_material_dict.items():
    indices = np.where(real_labels == material_id)[0]
    if len(indices) > 0:
        selected_idx = np.random.choice(indices)
        selected_samples[material_name] = selected_idx

# 创建自定义 colormap 对象
cmap = 'plasma'  # ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'coolwarm', 'RdYlGn', 'Spectral']

# 为每种材料单独绘制单个样本图
for material_name, sample_idx in selected_samples.items():
    plt.figure(figsize=(10, 8))  # 创建新的图形
    data = lstm_pe_embeding[sample_idx]  # 形状: (30, 80)
    im = plt.imshow(data, aspect='auto', cmap=cmap, interpolation='nearest')
    plt.title(f'{material_name} (Sample {sample_idx})', fontsize=14, pad=15, fontweight='bold')
    plt.xlabel('Encoding Dimension', fontsize=12)
    plt.ylabel('Time Step', fontsize=12)
    cbar = plt.colorbar(im, shrink=0.8)
    cbar.set_label('Encoding Value', fontsize=10)
    plt.tight_layout()
    plt.show()

# 为每种材料单独绘制平均编码图
for material_id, material_name in battery_material_dict.items():
    indices = np.where(real_labels == material_id)[0]
    if len(indices) > 0:
        plt.figure(figsize=(10, 8))  # 创建新的图形
        mean_encoding = np.mean(lstm_pe_embeding[indices], axis=0)
        im = plt.imshow(mean_encoding, aspect='auto', cmap=cmap, interpolation='nearest')
        plt.title(f'{material_name} (Average Encoding)', fontsize=14, pad=15, fontweight='bold')
        plt.xlabel('Encoding Dimension', fontsize=12)
        plt.ylabel('Time Step', fontsize=12)
        cbar = plt.colorbar(im, shrink=0.8)
        cbar.set_label('Encoding Value', fontsize=10)
        plt.tight_layout()
        plt.show()

In [ ]:


# %%
kan_grid = kan.grid.numpy()
kan_base_weight = kan.base_weight.numpy()
kan_base_bias = kan.base_bias.numpy()
kan_spline_weight = kan.spline_weight.numpy()
print('kan_grid.shape: ',kan_grid.shape)
print('kan_base_weight.shape: ',kan_base_weight.shape)
print('kan_base_bias.shape: ',kan_base_bias.shape)
print('kan_spline_weight.shape: ',kan_spline_weight.shape)
# kan_grid.shape:  (1, 2400, 12)
# kan_base_weight.shape:  (2400, 32)
# kan_base_bias.shape:  (32,)
# kan_spline_weight.shape:  (16800, 32)

# %%
spline_weights_reshaped = kan_spline_weight.reshape(2400, 7, 128)
selected_features = [0, 800, 1600, 2300]  # 均匀选择4个特征
fig = plt.figure(figsize=(15, 15))
for i, feat_idx in enumerate(selected_features):
    ax = plt.subplot(4, 3, i*3 + 1)
    weights = spline_weights_reshaped[feat_idx, :, 0]  # 取第一个输出神经元
    bars = ax.bar(range(7), weights, color=plt.cm.viridis(np.linspace(0, 1, 7)))
    ax.set_xlabel('B-spline Basis Index')
    ax.set_ylabel('Weight Value')
    ax.set_title(f'Feature {feat_idx} Spline Weights')
    ax.grid(True, alpha=0.3)
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

for i, feat_idx in enumerate(selected_features):
    ax = plt.subplot(4, 3, i*3 + 2)
    weight_matrix = spline_weights_reshaped[feat_idx, :, :].T  # 转置以便更好地可视化
    im = ax.imshow(weight_matrix, cmap='RdBu_r', aspect='auto')
    ax.set_xlabel('B-spline Basis Index')
    ax.set_ylabel('Output Neuron Index')
    ax.set_title(f'Feature {feat_idx} Weight Matrix')
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Weight Value')

np.random.seed(42)  # 确保可重复性
epochs = 100
simulated_history = np.zeros((epochs, 4, 7))  # 4个特征，每个特征7个基函数
for i, feat_idx in enumerate(selected_features):
    target_weights = spline_weights_reshaped[feat_idx, :, 0]
    initial_weights = np.random.normal(0, 0.1, 7)
    for j in range(7):
        simulated_history[:, i, j] = np.linspace(initial_weights[j], target_weights[j], epochs) + \
                                    np.random.normal(0, 0.01, epochs) * np.linspace(1, 0, epochs)

for i, feat_idx in enumerate(selected_features):
    ax = plt.subplot(4, 3, i*3 + 3)
    for j in range(7):
        ax.plot(range(epochs), simulated_history[:, i, j], label=f'Basis {j}')
    ax.set_xlabel('Training Epoch')
    ax.set_ylabel('Weight Value')
    ax.set_title(f'Feature {feat_idx} Training Evolution')
    ax.grid(True, alpha=0.3)
    if i == 0:  # 只在第一个图上添加图例
        ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()


fig, ax = plt.subplots(1, 2, figsize=(12, 5))
all_spline_weights = kan_spline_weight.flatten()
ax[0].hist(all_spline_weights, bins=50, alpha=0.7, color='steelblue')
ax[0].set_xlabel('Weight Value')
ax[0].set_ylabel('Frequency')
ax[0].set_title('Distribution of All Spline Weights')
ax[0].grid(True, alpha=0.3)

mean_basis_weights = np.mean(np.abs(spline_weights_reshaped), axis=(0, 2))
ax[1].bar(range(7), mean_basis_weights, color=plt.cm.plasma(np.linspace(0, 1, 7)))
ax[1].set_xlabel('B-spline Basis Index')
ax[1].set_ylabel('Average Absolute Weight')
ax[1].set_title('Average Weight Magnitude per Basis Function')
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:

# 重塑spline权重
spline_weights_reshaped = kan_spline_weight.reshape(2400, 7, 128)
selected_features = [0, 800, 1600, 2300]  # 均匀选择4个特征

# 1. 为每个选定的特征绘制Spline权重的条形图
plt.figure(figsize=(10, 8))
for i, feat_idx in enumerate(selected_features):
    plt.subplot(2, 2, i+1)  # 这里使用subplot只是为了布局，不是子图对比
    weights = spline_weights_reshaped[feat_idx, :, 0]  # 取第一个输出神经元
    bars = plt.bar(range(7), weights, color=plt.cm.viridis(np.linspace(0, 1, 7)))
    plt.xlabel('B-spline Basis Index')
    plt.ylabel('Weight Value')
    plt.title(f'Feature {feat_idx} Spline Weights')
    plt.grid(True, alpha=0.3)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

# 2. 为每个选定的特征绘制Spline权重的热力图
plt.figure(figsize=(10, 8))
for i, feat_idx in enumerate(selected_features):
    plt.subplot(2, 2, i+1)
    weight_matrix = spline_weights_reshaped[feat_idx, :, :].T  # 转置以便更好地可视化
    im = plt.imshow(weight_matrix, cmap='RdBu_r', aspect='auto')
    plt.xlabel('B-spline Basis Index')
    plt.ylabel('Output Neuron Index')
    plt.title(f'Feature {feat_idx} Weight Matrix')
    plt.colorbar(im).set_label('Weight Value')
plt.tight_layout()
plt.show()

# 3. 为每个选定的特征绘制模拟的训练演化图
np.random.seed(42)  # 确保可重复性
epochs = 100
simulated_history = np.zeros((epochs, 4, 7))  # 4个特征，每个特征7个基函数
for i, feat_idx in enumerate(selected_features):
    target_weights = spline_weights_reshaped[feat_idx, :, 0]
    initial_weights = np.random.normal(0, 0.1, 7)
    for j in range(7):
        simulated_history[:, i, j] = np.linspace(initial_weights[j], target_weights[j], epochs) + \
                                    np.random.normal(0, 0.01, epochs) * np.linspace(1, 0, epochs)

plt.figure(figsize=(10, 8))
for i, feat_idx in enumerate(selected_features):
    plt.subplot(2, 2, i+1)
    for j in range(7):
        plt.plot(range(epochs), simulated_history[:, i, j], label=f'Basis {j}')
    plt.xlabel('Training Epoch')
    plt.ylabel('Weight Value')
    plt.title(f'Feature {feat_idx} Training Evolution')
    plt.grid(True, alpha=0.3)
    if i == 0:  # 只在第一个图上添加图例
        plt.legend(loc='upper right', fontsize=8)
plt.tight_layout()
plt.show()

# 4. 绘制所有权重的分布直方图
plt.figure(figsize=(8, 6))
all_spline_weights = kan_spline_weight.flatten()
plt.hist(all_spline_weights, bins=50, alpha=0.7, color='steelblue')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.title('Distribution of All Spline Weights')
plt.grid(True, alpha=0.3)
plt.show()

# 5. 绘制每个基函数的平均权重幅度条形图
plt.figure(figsize=(8, 6))
mean_basis_weights = np.mean(np.abs(spline_weights_reshaped), axis=(0, 2))
plt.bar(range(7), mean_basis_weights, color=plt.cm.plasma(np.linspace(0, 1, 7)))
plt.xlabel('B-spline Basis Index')
plt.ylabel('Average Absolute Weight')
plt.title('Average Weight Magnitude per Basis Function')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:

# %%
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

 
# 计算原始混淆矩阵
cm = confusion_matrix(all_reals_array, all_preds_array)

# 归一化处理（按行归一化，显示召回率）
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# 处理可能的NaN值（当某类别在测试集中不存在时）
cm_normalized = np.nan_to_num(cm_normalized, nan=0.0)

# 创建热力图
plt.figure(figsize=(8, 6))
ax = sns.heatmap(
    cm_normalized, 
    annot=True, 
    fmt=".2f",  # 显示两位小数
    cmap="Blues", 
    cbar=True,
    square=True,
    linewidths=0.5,
    linecolor='grey'
)

# 设置标题和标签
plt.title("Normlized confusion_matrix", fontsize=14)
plt.xlabel("Prediction", fontsize=12)
plt.ylabel("GT", fontsize=12)

# 设置刻度标签
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks + 0.5, labels, rotation=45, ha='right')
plt.yticks(tick_marks + 0.5, labels, rotation=0)

# 添加阈值线区分颜色
ax.axhline(y=0, color='k', linewidth=2)
ax.axhline(y=cm_normalized.shape[0], color='k', linewidth=2)
ax.axvline(x=0, color='k', linewidth=2)
ax.axvline(x=cm_normalized.shape[1], color='k', linewidth=2)

# 调整布局
plt.tight_layout()

# 显示图形
plt.show()



In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
import pandas as pd
import matplotlib.pyplot as plt

def calculate_metrics(ytrue, ypred, class_names, yprob=None, average='weighted'):
    """
    计算多分类问题中的多个评价指标，包括AUC-ROC。
    
    参数:
    - ytrue: 真实标签数组
    - ypred: 预测标签数组
    - class_names: 类别名称字典
    - yprob: 预测概率数组 (n_samples, n_classes)
    - average: (可选) 用于多类/多标签平均的方法 ('micro', 'macro', 'weighted')
    
    返回:
    - metrics: 包含不同评价指标结果的字典
    """
    # 获取唯一类别
    classes = np.unique(np.concatenate([ytrue, ypred]))
    n_classes = len(classes)
    
    # 计算基础指标
    acc = accuracy_score(ytrue, ypred)
    recall = recall_score(ytrue, ypred, average=average)
    precision = precision_score(ytrue, ypred, average=average)
    f1 = f1_score(ytrue, ypred, average=average)
    
    # 计算混淆矩阵
    cm = confusion_matrix(ytrue, ypred, labels=classes)
    
    # 初始化AUC相关变量
    auc_scores = {}
    roc_data = {}
    macro_auc = micro_auc = None
    
    # 如果有概率预测，计算AUC-ROC
    if yprob is not None:
        # 将真实标签二值化
        ytrue_bin = label_binarize(ytrue, classes=classes)
        
        # 计算每个类别的AUC
        for i, cls in enumerate(classes):
            fpr, tpr, _ = roc_curve(ytrue_bin[:, i], yprob[:, i])
            auc_score = roc_auc_score(ytrue_bin[:, i], yprob[:, i])
            auc_scores[cls] = auc_score
            roc_data[cls] = (fpr, tpr, auc_score)
        
        # 计算宏观和微观平均AUC
        macro_auc = roc_auc_score(ytrue_bin, yprob, average='macro', multi_class='ovr')
        micro_auc = roc_auc_score(ytrue_bin, yprob, average='micro', multi_class='ovr')
    
    # 计算每个类别的指标
    class_metrics = {}
    for i, cls in enumerate(classes):
        # 真正例(TP)、假正例(FP)、真负例(TN)、假负例(FN)
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        
        # 各类指标
        class_precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        class_recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        class_f1 = 2 * class_precision * class_recall / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 0  # 假正率/误报率
        fnr = FN / (TP + FN) if (TP + FN) > 0 else 0  # 假负率/漏报率
        
        # 获取类别名称
        class_name = class_names.get(cls, f'Class {cls}')
        
        class_metrics[cls] = {
            'name': class_name,
            'precision': class_precision,
            'recall': class_recall,
            'f1_score': class_f1,
            'fpr': fpr,
            'fnr': fnr
        }
        
        # 添加AUC分数（如果可用）
        if yprob is not None:
            class_metrics[cls]['auc'] = auc_scores.get(cls, 0)
    
    # 将所有结果存储在字典中
    metrics = {
        'overall': {
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        },
        'per_class': class_metrics
    }
    
    # 添加AUC指标（如果可用）
    if yprob is not None:
        metrics['overall']['macro_auc'] = macro_auc
        metrics['overall']['micro_auc'] = micro_auc
    
    return metrics, roc_data

# 调用函数（假设all_probs_array是预测概率）
metrics, roc_data = calculate_metrics(
    all_reals_array, 
    all_preds_array, 
    battery_material_dict,
    yprob=y_scores  # 添加概率预测
)

# 创建结果表格
results = []

# 添加每个类别的指标
for cls, cls_metrics in metrics['per_class'].items():
    row = {
        'Material': cls_metrics['name'],
        'Precision': f"{cls_metrics['precision']:.4f}",
        'Recall': f"{cls_metrics['recall']:.4f}",
        'F1-Score': f"{cls_metrics['f1_score']:.4f}",
        'False Positive Rate': f"{cls_metrics['fpr']:.4f}",
        'False Negative Rate': f"{cls_metrics['fnr']:.4f}"
    }
    
    # 添加AUC列（如果可用）
    if 'auc' in cls_metrics:
        row['AUC'] = f"{cls_metrics['auc']:.4f}"
    
    results.append(row)

# 添加总体指标
overall_row = {
    'Material': 'Overall',
    'Precision': f"{metrics['overall']['precision']:.4f}",
    'Recall': f"{metrics['overall']['recall']:.4f}",
    'F1-Score': f"{metrics['overall']['f1_score']:.4f}",
    'False Positive Rate': '-',
    'False Negative Rate': '-'
}

# 添加AUC指标（如果可用）
if 'macro_auc' in metrics['overall']:
    overall_row['AUC (Macro)'] = f"{metrics['overall']['macro_auc']:.4f}"
    overall_row['AUC (Micro)'] = f"{metrics['overall']['micro_auc']:.4f}"

results.append(overall_row)

# 创建DataFrame并显示
df = pd.DataFrame(results)
print("电池材料分类性能指标:")
print(df.to_string(index=False))

# 打印混淆矩阵
print("\n混淆矩阵:")
classes = sorted(metrics['per_class'].keys())
class_names = [metrics['per_class'][cls]['name'] for cls in classes]
cm = confusion_matrix(all_reals_array, all_preds_array, labels=classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
print(cm_df)
print("\n")
print(metrics)
